In [80]:
import pickle 
import tensorflow as tf 
from tensorflow import keras 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import random

In [81]:
def loadModel(modelName):
    model = pickle.load(open(modelName, 'rb'))
    return model 

 
def getDataset(dataFolder, subset, imageSize = (224, 224), batchSize = 32):
    train_ds = keras.utils.image_dataset_from_directory(
      dataFolder,
      seed=123,
      image_size=imageSize,
      batch_size=batchSize)
    return train_ds


In [82]:
EPOCHS = 5
modelName = "efficientNetB5"
trainingFolder = 'data/x5/train/RGB/'
testingFolder = 'data/x5/test_with_labels/RGB/'
# IMPORTANT
train_ds = getDataset(trainingFolder, "training", imageSize=(456, 456), batchSize=32)
val_ds =  getDataset(testingFolder, "validation", imageSize=(456, 456), batchSize=32)

Found 6000 files belonging to 7 classes.
Found 1500 files belonging to 7 classes.


In [102]:
# model = loadModel("model_efficientNetB0_phase2.pkl")
# ERROR: make sure that we have the space b/w the _ and the 
import os, sys

class Model: 
    def __init__(self, name, path, imageSize, batchSize):
        self.name = name 
        self.path = path 
        self.testingFolder = 'data/x5/test_with_labels/RGB/'
        self.imageSize = imageSize
        self.batchSize = batchSize
    
    def loadModel(self):
        pickledModel = pickle.load(open(self.path, 'rb'))
        return pickledModel
    
    def loadValidationDataset(self):
        val_ds = getDataset(self.testingFolder, "validation", imageSize=self.imageSize, batchSize=self.batchSize)
        return val_ds
    
    def getAccuracy(self):
        model = self.loadModel()
        val_ds = self.loadValidationDataset()
        score = model.evaluate(val_ds)
        print("Name:\t", self.name)
        print('Test loss:\t', score[0]) 
        print('Test accuracy:\t', score[1])
    
    def getMetrics(self): 
        model = self.loadModel()
        val_ds = self.loadValidationDataset()
        pred = model.predict(val_ds)
        predLabels = tf.argmax(pred, axis=1)
        testLabels = np.concatenate([y for x, y in val_ds], axis=0)
        classNames = val_ds.class_names
        print("Name:\t", self.name)
        print("F1 Score:\t", f1_score(testLabels, predLabels, average="macro"))
        print("Precision Score:\t", precision_score(testLabels, predLabels, average="macro"))
        print("Recall Score:\t", recall_score(testLabels, predLabels, average="macro"))    


def getModels(): 
    path = "models/"
    dirs = os.listdir(path)
    isNotFile = lambda x: x[0] == "."
    models = []
    for file in dirs:
        if isNotFile(file):
            continue 
        _, name, phase = file.split("_")
        if phase.strip() == "phase2.pkl": 
            name = name.lower()
            imageSize = (224, 224)  #default for B0, DenseNet & ResNet
            batchSize = 32
            if name == "efficientnetb5": 
                imageSize = (456, 456)
            elif name == "efficientnetb7":
                imageSize = (600, 600)
            elif name == "resnet50":
                batchSize = 64
            model = Model(name.upper(), path+file, imageSize, batchSize)
            models.append(model)          
    return models 
    

In [103]:
# Load up the models as objects
models = getModels()

In [105]:
for model in models: 
    model.getMetrics()
        


KeyboardInterrupt



In [106]:
for model in models: 
    model.getAccuracy()

Found 1500 files belonging to 7 classes.


2022-03-07 18:51:43.890113: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - 21s 573ms/step - loss: 1.3541 - accuracy: 0.7273
Name:	 RESNET50
Test loss:	 1.3541253805160522
Test accuracy:	 0.7273333072662354
Found 1500 files belonging to 7 classes.


2022-03-07 18:53:08.516217: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


47/47 [==============================] - 52s 1s/step - loss: 1.1829 - accuracy: 0.7160
Name:	 EFFICIENTNETB5
Test loss:	 1.1828933954238892
Test accuracy:	 0.7160000205039978
Found 1500 files belonging to 7 classes.


2022-03-07 18:55:01.890352: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


47/47 [==============================] - 14s 242ms/step - loss: 1.6415 - accuracy: 0.7100
Name:	 EFFICIENTNETB0
Test loss:	 1.641497254371643
Test accuracy:	 0.7099999785423279
Found 1500 files belonging to 7 classes.


2022-03-07 18:56:16.928797: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


47/47 [==============================] - 22s 395ms/step - loss: 1.8497 - accuracy: 0.6680
Name:	 DENSENET121
Test loss:	 1.849739909172058
Test accuracy:	 0.6679999828338623


F1 Score:	 0.1329745530552005
Precision Score:	 0.13471223159575937
Recall Score:	 0.13719080561271843


In [57]:
from hyperas import optim
from hyperopt import Trials, STATUS_OK, tpe


best_run = optim.minimize(model=model,
                          data=(train_ds, val_ds),
                          algo=tpe.suggest,
                          max_evals=10,
                          trials=Trials())

TypeError: module, class, method, function, traceback, frame, or code object was expected, got Functional